In [405]:
from pickle import load as pickle_load
from numpy import array as np_array
from pandas import DataFrame as pd_DataFrame
from pandas import cut as pd_cut
from enum import Enum

In [406]:
#С предсказаниями кбм точно все хорошо с любым форматом 1 и '1'
#knr все ок если формат [0, 1, 5, 2, 0, 'wd']
#lgbmr все ок если формат [1, 2, 3, 4, 5, 'wd]

In [407]:
with open('ohe', 'rb') as f: #one hot encoder
    ohe = pickle_load(f) 

with open('cmbmodel', 'rb') as f: #catboost
    cbm = pickle_load(f)

with open('lgbm', 'rb') as f:
    lgbm = pickle_load(f)  #lgbm

with open('knr', 'rb') as f:
    knr = pickle_load(f)  #kn regressor

with open('rfrblend', 'rb') as f:
    rfrblend = pickle_load(f) #три предикта от предыдущих моделей на вход

In [416]:
#Можно в принципе просто через cbm.predict([1, 2, 3, 4, 5, 'wd']) взаимодейстовать

with open('cbm7', 'rb') as f:
    cbm7 = pickle_load(f) #тот же вход, что в cmb + среднее за последние 7д в конце

wtl работает с указанием столбца <br/>
ttb работает просто входом, но на одном примере <br/>
tocat работает просто с входом на всей выборке <br/>
ohe работает на всей выборке <br/>


In [409]:
def weekday_to_label(series, weekdayz = 'Ru'): #можно задать и другой enum, исправив функцию

    """
        Список вида ['пн', 'вт', 'ср', 'вс'] превращает в [1, 2, 3, 7]
        Необходимо передать список и вид преобразования или собственную функцию 
        'Ru' - классические ['пн', 'вт', 'ср'] -> [1, 2 , 3]
        'Eng' - ['sun', 'mon', 'tue'] -> [1, 2, 3]
        'Eng1' - ['mon', 'tue', 'wed'] -> [1, 2, 3]
        """

    x = 0
    if weekdayz == 'Ru':
        weekdayz = Enum('weekday numeration', 'пн вт ср чт пт сб вс')

    if weekdayz == 'Eng1':
        weekdayz = Enum('weekday numeration', 'mon tue wed thu fri sat sun')

    if weekdayz == 'Eng':
        weekdayz = Enum('weekday numeration', 'sun mon tue wed thu fri sat')

        

    try:
        if len(series) > 2:
            try:
                for each in range(0, len(series)):
                    series[each] = weekdayz[series[each].lower()]._value_ #изначально была конструкция 
                    x =+ 1                   #вида df['День недели'][each] = weekdayz[df['День недели'][each].lower()]._value_
            
            except KeyError:
                print('KeyError ошибка \n Индексы должны быть по порядку, иначе не работает')

            except AttributeError:
                print('AttributeError ошибка \n Возможно, часть уже преобразована в цифры \n Не умею работать с цифрами')
        if len(series) == 2:
            return weekdayz[series.lower()]._value_ #возможно нужно будет поправить
    except:
        print('Неверные данные')

In [410]:
def temp_to_bins(day_params): #для работоспособности нужен pd.cut(), не работает с списком дней, у меня не получилось
    day_params[3] = int(day_params[3])
    bins = [-50, 0, 15, 25, 50]
    labels = [1, 2, 3, 4]
    if day_params[3] > -50 and day_params[3] < 50:
        day_params[3] = pd_cut([day_params[3]], bins=bins, labels=labels)[0]
        return day_params
    else:
        print('Измените температуру на реалистичную')

In [411]:
def to_category(day_params): 
    for col in day_params.select_dtypes(include=['int64', 'object']):
        day_params[col] = day_params[col].astype('category')


***

In [412]:
def predictpls(day_variables):
# day_variables = np_array(day_variables)
   
   #Общая предобработка
   day_variables[-1] = weekday_to_label(day_variables[-1])
   day_variables = temp_to_bins(day_variables)
   df = pd_DataFrame(columns=['Девочка', 'Мальчик', 'Месяц', 'Температура', 'Дождь', 'День недели'])
   df.loc[0] = day_variables

   x2 = cbm.predict(day_variables) #тут может выдавать предсказания cbm

   #предобработка для knr
   df1 = df.copy()
   df1 = ohe.transform(df1)
   x1 = knr.predict(df1)

   #предобработка для lgbm
   to_category(df)
   x3 = lgbm.predict(df)



   data = {'x1':[x1], 'x2':[x2], 'x3':[x3]}
   return rfrblend.predict(pd_DataFrame(data=data))

In [415]:
predictpls([0, 1, 6, 28, 0, 'ср'])

array([5711.54370629])